In [9]:
from neural_nlp.benchmarks import benchmark_pool
pereira = benchmark_pool["Pereira2018-encoding"]
data = pereira._load_assembly()

data

<xarray.NeuroidAssembly (presentation: 627, neuroid: 103900)>
array([[-0.61048431, -0.76491186, -0.79946189, ..., -1.05059589,
        -0.87116667, -1.60948908],
       [ 1.74291344,  2.00353428,  1.97103177, ...,  1.68688988,
         1.4843575 ,  1.88385042],
       [-0.57701107, -0.24646438, -0.28553028, ...,  0.12177334,
        -0.87131264, -1.22506831],
       ...,
       [ 0.4911479 ,  0.97394189,  0.14704561, ...,  2.23514024,
         1.51163703,  0.58961535],
       [ 1.0331004 ,  1.5348565 ,  0.84328902, ...,  4.25791121,
         2.44102726,  3.44806095],
       [ 0.53970481,  0.98636439,  0.53409886, ...,  7.50497969,
         2.68749469,  4.16021721]])
Coordinates:
  * presentation      (presentation) MultiIndex
  - stimulus_num      (presentation) int64 0 0 1 1 2 2 ... 379 380 381 382 383
  - passage_index     (presentation) int64 1 1 1 1 1 1 1 ... 95 95 96 96 96 96
  - passage_label     (presentation) object 'Accordion' 'beekeeping' ... 'Woman'
  - passage_category  (presentation) object 'music' 'beekeeping' ... 'human'
  - stimulus_id       (presentation) object '384sentences.0' ... '384sentence...
  - story             (presentation) object '384sentences.music' ... '384sent...
  - experiment        (presentation) object '384sentences' ... '384sentences'
  * neuroid           (neuroid) MultiIndex
  - subject           (neuroid) object '018' '018' '018' ... '426' '426' '426'
  - voxel_num         (neuroid) int64 28 29 31 32 38 ... 32539 32540 32607 32611
  - atlas             (neuroid) object 'language' 'language' ... 'MD' 'MD'
  - filter_strategy   (neuroid) object '' '' '' ... 'HminusE' 'HminusE'
  - atlas_selection   (neuroid) object 'from90to100prcnt' ... 'from90to100prcnt'
  - roi               (neuroid) object 'LH_AntTemp' ... 'LH_supFrontal'
  - indices_in_3d     (neuroid) int64 72505 72506 72584 ... 454542 461888 461967
  - col_to_coord_1    (neuroid) int64 62 63 62 63 63 63 63 ... 54 55 54 55 54 54
  - col_to_coord_2    (neuroid) int64 63 63 64 64 65 66 63 ... 53 53 54 54 52 53
  - col_to_coord_3    (neuroid) int64 10 10 10 10 10 10 11 ... 61 61 61 61 62 62
  - neuroid_id        (neuroid) object '018.28' '018.29' ... '426.32611'
Attributes:
    version:            base
    stimulus_set:                                                     sentenc...
    class_module:       brainio_base.assemblies
    class_name:         NeuroidAssembly
    stimulus_set_name:  Pereira2018

In [10]:
# post-processing copied from PereiraBenchmark in neural.py (lines 393/394)

data = data.dropna('neuroid')

print(data.shape)

(627, 49760)


In [14]:
import numpy as np

# Extract stimuli (sentences) from xarray
stimuli_texts = {}
for _, row in data.attrs['stimulus_set'].iterrows():
    stimuli_texts[row['stimulus_id']] = row['sentence']


# Extract fmri data
stimuli = []
brain_reps = []
for idx, stimulus_id in enumerate(data.coords['stimulus_id']):
    stimuli.append(stimuli_texts[stimulus_id.item()])
    brain_reps.append(data.values[idx, :])

brain_reps = np.array(brain_reps)

assert len(stimuli) == len(brain_reps)

print(brain_reps.shape)

(627, 49760)


In [13]:
# import pickle
# from pathlib import Path
#
# corpora = Path(__file__) / "../corpora/"
# corpora.mkdir(exist_ok=True)
#
# pereira_datafile = corpora / "pereira.pkl"
#
# with open(pereira_datafile, 'wb') as f:
#     pickle.dump((stimuli, brain_reps), f)

In [ ]:
# start running the notebook here if you 
# have the pereira.pkl file already!

# import pickle
# import numpy as np
#
#
#
# with open('../corpora/pereira.pkl', 'rb') as f:
#     stimuli, brain_reps = pickle.load(f)
#
# print(brain_reps.shape)

In [15]:
import torch
from transformers import GPT2Tokenizer, GPT2Model

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2', output_hidden_states=True)
model = model.eval()  # Turn off dropout

model

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0): GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (1): GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP

In [16]:
import numpy as np
from tqdm import tqdm

hidden_states = []
with torch.no_grad():
    for stimulus in tqdm(stimuli):
        output = model(
            **tokenizer([stimulus], add_special_tokens=True, return_tensors='pt')
        )
        
        layer_reps = []
        for layer_hidden_states in output.hidden_states:
            # in models/implementations.py, Transformer uses the rep
            # of the last word (line 595)
            layer_reps.append(layer_hidden_states.squeeze()[-1, :].numpy())
        hidden_states.append(layer_reps)
hidden_states = np.array(hidden_states)
print(hidden_states.shape)

100%|██████████| 627/627 [00:19<00:00, 32.22it/s]

(627, 13, 768)


In [19]:
from scipy.stats import pearsonr
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

k_folds = KFold(n_splits=5, shuffle=True)

pearsonrs = []
for fold, (train_indices, test_indices) in enumerate(k_folds.split(hidden_states)):
    pearsonrs.append([])
    train_brain_reps, test_brain_reps = brain_reps[train_indices], brain_reps[test_indices]
    for layer_num in tqdm(range(len(hidden_states[0])), desc='fold%s' % fold):
        train_hidden_states, test_hidden_states = \
            hidden_states[train_indices, layer_num, :], \
            hidden_states[test_indices, layer_num, :]


        # TODO: Are they doing any kind of hyperparameter tuning
        # (regularization, etc) here?  We're using SKLearn's defaults

        model = LinearRegression().fit(train_hidden_states, train_brain_reps)
        pred_brain_reps = model.predict(test_hidden_states)
        
        # We aggregated voxel/electrode/ROI predictivity scores by taking the
        # median of scores for each participant’s voxels/electrodes/ROIs and
        # then computing the median across participants. Finally, this score was
        # divided by the estimated ceiling value (see below) to yield a final score in
        # the range [0, 1].
        
        # https://github.com/brain-score/brain-score/blob/master/brainscore/metrics/xarray_utils.py#L78
        # https://github.com/brain-score/brain-score/blob/master/brainscore/metrics/regression.py#L33
        # https://github.com/brain-score/brain-score/blob/master/brainscore/metrics/transformations.py#L42
        
        # not totally sure this is right...
        
        layer_pearson_rs = []
        for pred_brain_rep, test_brain_rep in zip(pred_brain_reps, test_brain_reps):
            layer_pearson_rs.append(pearsonr(pred_brain_rep, test_brain_rep))
        
        pearsonrs[-1].append(np.median(layer_pearson_rs))


fold4: 100%|██████████| 13/13 [03:56<00:00, 18.18s/it]


In [20]:
pearsonrs = np.array(pearsonrs)
for layer_num in range(pearsonrs.shape[1]):
    print((layer_num, np.mean(pearsonrs[:, layer_num]), np.median(pearsonrs[:, layer_num])))


(0, 0.00408277367472118, 8.939323940392131e-14)
(1, 1.422616914515031e-15, 1.8549810867016208e-22)
(2, 6.320003546318065e-08, 6.83100467308954e-22)
(3, 9.14089075647834e-14, 5.9366842767137336e-21)
(4, 8.801125140245791e-07, 3.5496849966057153e-15)
(5, 4.2953452545340317e-19, 1.8873712591467982e-27)
(6, 2.3607197244805418e-12, 3.500090920938019e-24)
(7, 2.917129875655254e-08, 2.6814374774741727e-11)
(8, 3.231244327004638e-11, 1.435822032253782e-14)
(9, 1.2707904948769912e-06, 6.587829655035209e-15)
(10, 8.891380065260758e-18, 1.3435720055964806e-22)
(11, 1.8299568950772305e-17, 1.6172832278772864e-20)
(12, 7.724724866125182e-11, 1.0161555789294238e-19)


In [22]:
# Compare with the ceiling
pereira.ceiling

<xarray.Score (aggregation: 3)>
array([0.31856696, 0.01295766, 0.01657364])
Coordinates:
  * aggregation  (aggregation) object 'center' 'error_low' 'error_high'
Attributes:
    class_module:  brainscore.metrics
    class_name:    Score